# Assignment 5

Build CNN model for sentiment analysis (binary classification) of IMDB Reviews (https://www.kaggle.com/utathya/imdb-review-dataset). You can use data with label="unsup" for pretraining of embeddings. Here you are forbidden to use test dataset for pretraining of embeddings.
Your quality metric is accuracy score on test dataset. Look at "type" column for train/test split.
You can use pretrained embeddings from external sources.
You have to provide data for trials with different hyperparameter values.

You have to beat following baselines:
[3 points] acc = 0.75
[5 points] acc = 0.8
[8 points] acc = 0.9

[2 points] for using unsupervised data

In [0]:
import pandas as pd

import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from collections import Counter
import itertools

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

from sklearn import model_selection 
from sklearn.metrics import accuracy_score

In [2]:
tt.cuda.is_available()

True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving imdb-review-dataset.zip to imdb-review-dataset (1).zip


In [4]:
!unzip imdb-review-dataset.zip

Archive:  imdb-review-dataset.zip
replace imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: imdb_master.csv         


In [5]:
data = pd.read_csv('imdb_master.csv', encoding='latin-1')
data = data.drop('Unnamed: 0', axis = 1)
data

,type,review,label,file
0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt
...,...,...,...,...
99995,train,"Delightfully awful! Made by David Giancola, a ...",unsup,9998_0.txt
99996,train,"Watching Time Chasers, it obvious that it was ...",unsup,9999_0.txt
99997,train,At the beginning we can see members of Troma t...,unsup,999_0.txt
99998,train,"The movie was incredible, ever since I saw it ...",unsup,99_0.txt


In [0]:
clean_data = data[(data['type'] == 'train') & (data['label'] != 'unsup')]
withunsup_data  =   data[(data['type'] == 'train') ]
test_data = data[(data['type'] == 'test')]
train_data, val_data = model_selection.train_test_split(clean_data, test_size=0.05, stratify=clean_data.label)

In [0]:
class InputFeatures(object): 
    def __init__(self, input_ids, label_id):
        self.input_ids = input_ids
        self.label_id = label_id

In [0]:
class Vocab:
    def __init__(self, itos, unk_index):
        self._itos = itos
        self._stoi = {word:i for i, word in enumerate(itos)}
        self._unk_index = unk_index
        
    def __len__(self):
        return len(self._itos)
    
    def word2id(self, word):
        idx = self._stoi.get(word)
        if idx is not None:
            return idx
        return self._unk_index
    
    def id2word(self, idx):
        return self._itos[idx]

In [0]:
class PreProc():
    def __init__(self, max_vocab_size):
        self.special_words = ['<PAD>', '</UNK>', '<S>', '</S>']
        self.unk_index = 1
        self.pad_index = 0
        self.vocab = None
        self.max_vocab_size = max_vocab_size
        
    def tokenize(self, text):
        text = re.sub(r'[^\w\s]','',text, re.UNICODE)
        text = text.lower()
        text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
        text = [lemmatizer.lemmatize(token, "v") for token in text]
        text = [word for word in text if word not in en_stopwords]
        return text
        
    def build_vocab(self, tokens):
        itos = []
        itos.extend(self.special_words)
        
        token_counts = Counter(tokens)
        for word, _ in token_counts.most_common(self.max_vocab_size - len(self.special_words)):
            itos.append(word)
            
        self.vocab = Vocab(itos, self.unk_index)
    
    def transform(self, texts):
        result = []
        for text in texts:
            tokens = ['<S>'] + self.tokenize(text) + ['</S>']
            ids = [self.vocab.word2id(token) for token in tokens]
            result.append(ids)
        return result
    
    def fit_transform(self, texts):
        result = []
        tokenized_texts = [self.tokenize(text) for text in texts]
        self.build_vocab(itertools.chain(*tokenized_texts))

        for tokens in tokenized_texts:
            tokens = ['<S>'] + tokens + ['</S>']
            ids = [self.vocab.word2id(token) for token in tokens]
            result.append(ids)

        return result

In [0]:
def build_features(token_ids, label, max_seq_len, pad_index, label_encoding):
    if len(token_ids) >= max_seq_len:
        ids = token_ids[:max_seq_len]
    else:
        ids = token_ids + [pad_index for _ in range(max_seq_len - len(token_ids))]
    return InputFeatures(ids, label_encoding[label])


def features_to_tensor(features):
    text_tensor = tt.tensor([example.input_ids for example in features], dtype=tt.long)
    labels_tensor = tt.tensor([example.label_id for example in features], dtype=tt.long)
    return text_tensor, labels_tensor

In [0]:
max_seq_len=200
classes = {'neg': 0, 'pos' : 1}
text2id = PreProc(10000)

train_ids = text2id.fit_transform(train_data['review'])
val_ids = text2id.transform(val_data['review'])
test_ids = text2id.transform(test_data['review'])

In [0]:
train_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(train_ids, train_data['label'])]

val_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(val_ids, val_data['label'])]

test_features = [build_features(token_ids, label,max_seq_len, text2id.pad_index, classes) 
                  for token_ids, label in zip(test_ids, test_data['label'])]

In [0]:
batch_size = 64

In [0]:
train_tensor, train_labels = features_to_tensor(train_features)
val_tensor, val_labels = features_to_tensor(val_features)
test_tensor, test_labels = features_to_tensor(test_features)

train_dataset = TensorDataset(train_tensor, train_labels)
val_dataset = TensorDataset(val_tensor, val_labels)
test_dataset = TensorDataset(test_tensor, test_labels)

train_loader = DataLoader(train_dataset, batch_size = batch_size)
val_loader = DataLoader(val_dataset, batch_size = batch_size)
test_loader = DataLoader(test_dataset, batch_size = batch_size)

In [0]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.preproc = nn.Sequential(
            nn.Embedding(10000,50)
        )
        self.hidden = nn.Sequential(
            nn.Conv1d(in_channels=50, out_channels=60, kernel_size=3), 
            nn.ReLU(), nn.MaxPool1d(3,stride=2),      
            nn.Conv1d(in_channels=60, out_channels=100, kernel_size=3), 
            nn.ReLU(), nn.MaxPool1d(3, stride=2))
        
        self.output = nn.Sequential(
            nn.Linear(4700,1),
            nn.Sigmoid()
        )
    def forward(self, x):
        batch = x.size(0)
        x = self.preproc(x)
        x = x.transpose(2,1)
        
        y = self.hidden(x).view(batch, -1)
        return  self.output(y)

In [0]:
def fit(net,criterion,train_loader,val_loader,optimizer, epochs):
    best=0
    net.cuda()

    for i in range(epochs):
        tr_loss = 0
        val_loss = 0
        val_accuracy =0

        for xx,yy in train_loader:
            xx, yy = xx.cuda(), yy.cuda()
            optimizer.zero_grad()
            y = net.forward(xx)
            loss = criterion(y,yy.float().view(len(yy),-1))
            tr_loss += loss
            loss.backward()
            optimizer.step()

        tr_loss /= len(train_loader)
    
        with tt.no_grad():
            for xx,yy in val_loader:
    
                all_preds = []
                xx, yy = xx.cuda(), yy.cuda()
                y = net.forward(xx)
                loss = criterion(y,yy.float().view(len(yy),-1))
                val_loss += loss
  
                for index in y:
                    if index>0.5:
                        all_preds.append(1)
                    else:
                        all_preds.append(0)
      
                yy = yy.cpu().numpy()
                val_accuracy += accuracy_score(all_preds,yy)

            val_accuracy /= len(val_loader)
            if val_accuracy>best:
                best = val_accuracy
                tt.save(net.state_dict(), "../model.py")

        print((f"epoch: {i}, train: {tr_loss.item()}, val: {val_accuracy.item()}"))
    net.cpu()

In [17]:
model = MyModel()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)
fit(model,criterion,train_loader,val_loader,optimizer,70)

epoch: 0, train: 0.5948524475097656, val: 0.7797334558823529
epoch: 1, train: 0.38715213537216187, val: 0.8418658088235293
epoch: 2, train: 0.29508525133132935, val: 0.8443933823529411
epoch: 3, train: 0.23098213970661163, val: 0.8410845588235294
epoch: 4, train: 0.17585469782352448, val: 0.8520220588235293
epoch: 5, train: 0.15666772425174713, val: 0.8388327205882353
epoch: 6, train: 0.1455235779285431, val: 0.8448988970588236
epoch: 7, train: 0.11544328182935715, val: 0.8370863970588236
epoch: 8, train: 0.07831399887800217, val: 0.8550551470588236
epoch: 9, train: 0.05606941506266594, val: 0.8573988970588236
epoch: 10, train: 0.051253173500299454, val: 0.8612132352941175
epoch: 11, train: 0.04713822528719902, val: 0.8613051470588236
epoch: 12, train: 0.04420037567615509, val: 0.8626838235294118
epoch: 13, train: 0.016389163210988045, val: 0.8573069852941175
epoch: 14, train: 0.005285417661070824, val: 0.8566176470588236
epoch: 15, train: 0.002388832625001669, val: 0.8627757352941176


In [0]:
def eval(test_loader):
  y_pred = []
  y_true = []
  with tt.no_grad():
    model.eval()
    for texts, labels in test_loader:
        model.cuda()
        texts = texts.cuda()
        output = model.forward(texts)
        for i in output:
            if i>0.5:
                y_pred.append(1)
            else:
                y_pred.append(0)
        y_true.extend(labels.tolist())
  return accuracy_score(y_pred, y_true)

In [19]:
eval(test_loader)

0.83996